In [6]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the dataset
dataset = tf.keras.utils.image_dataset_from_directory(
    'C:/Users/vedant/Downloads/archive (1)\data/food-101-tiny/valid',
    batch_size=32,
    image_size=(180, 180)
)

# Get class names
class_names = dataset.class_names

Found 500 files belonging to 10 classes.


In [7]:
calorie_mapping = {
    'apple_pie': 237,
    'bibimbap': 560,
    'cannoli': 230,
    'edamame': 122,
    'falafel': 333,
    'french_toast': 229,
    'ice_cream': 273,
    'ramen': 436,
    'sushi': 306,
    'tiramisu': 492
}
calorie_values = np.array([calorie_mapping[name] for name in class_names])

In [8]:

import tensorflow as tf

import numpy as np

# Assuming you have already defined calorie_mapping and class_names

# Convert calorie_mapping to a TensorFlow lookup table
calorie_table = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant(list(calorie_mapping.keys())),
        values=tf.constant(list(calorie_mapping.values()), dtype=tf.float32)
    ),
    default_value=tf.constant(0.0)
)

def preprocess(images, labels):
    images = tf.keras.applications.mobilenet_v2.preprocess_input(images)
    
    # Convert class names to calorie values
    calorie_values = calorie_table.lookup(tf.gather(class_names, labels))
    
    return images, calorie_values

# Apply the preprocessing to your dataset
preprocessed_dataset = dataset.map(preprocess)

# Split the dataset
train_ds = preprocessed_dataset.take(int(len(dataset) * 0.8))
val_ds = preprocessed_dataset.skip(int(len(dataset) * 0.8))

# Configure the dataset for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [9]:
dataset = tf.keras.utils.image_dataset_from_directory(
    'C:/Users/vedant/Downloads/archive (1)\data/food-101-tiny/valid',
    batch_size=32,
    image_size=(224, 224)  # Changed from (180, 180)
)

# Then update your model input shape
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

Found 500 files belonging to 10 classes.


In [10]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(180, 180, 3),
    include_top=False,
    weights=None  # Initialize without pre-trained weights
)

In [11]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(180, 180, 3),
    include_top=False,
    weights=None  # Initialize without pre-trained weights
)

In [12]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer for calorie prediction
])

model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 77s 4s/step - loss: 117214.7812 - mae: 315.3896 - val_loss: 123316.1406 - val_mae: 323.2070
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 46s 4s/step - loss: 89826.8906 - mae: 271.3820 - val_loss: 122154.6719 - val_mae: 321.4052
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 44s 4s/step - loss: 50477.2617 - mae: 193.6122 - val_loss: 118789.4688 - val_mae: 316.1267
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 44s 4s/step - loss: 23705.7598 - mae: 125.4270 - val_loss: 107232.4766 - val_mae: 297.2863
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 43s 4s/step - loss: 11638.7334 - mae: 83.2422 - val_loss: 94927.9922 - val_mae: 275.8163
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 45s 4s/step - loss: 11874.6172 - mae: 91.5394 - val_loss: 84529.9062 - val_mae: 256.2744
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 45s 4s/step - loss: 8073.4272 - mae: 68.0691 - val_loss: 77805.1562 - val_mae: 242.7999
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 45s 4s/step - loss: 5945.9092 - mae: 56.6370 - val_loss: 78529.94

In [13]:
import tensorflow as tf

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.MeanAbsoluteError()]
)

In [18]:
import os
import numpy as np
import tensorflow as tf

test_dir = 'C:/Users/vedant/Downloads/archive (1)\data/food-101-tiny/valid'
test_images = []
test_labels = []

for food_item in os.listdir(test_dir):
    if food_item in calorie_mapping:
        img_path = os.path.join(test_dir, food_item)
        test_images.append(img_path)
        test_labels.append(food_item)

def load_and_preprocess_image(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis
    return tf.keras.applications.mobilenet_v2.preprocess_input(img_array)

test_data = [load_and_preprocess_image(img_path) for img_path in test_images]
test_data = np.vstack(test_data)

PermissionError: [Errno 13] Permission denied: 'C:/Users/vedant/Downloads/archive (1)\\data/food-101-tiny/valid\\apple_pie'